In [ ]:
!pip install transformers
!pip install datasets

#### modules required for aesthetic scores

In [ ]:
!pip install webdataset

In [ ]:
!pip install pytorch_lightning

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
# download script for aesthetic score
# !wget https://raw.githubusercontent.com/microsoft/LMOps/main/promptist/aesthetic/simple_inference.py

--2023-10-04 03:04:08--  https://raw.githubusercontent.com/microsoft/LMOps/main/promptist/aesthetic/simple_inference.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2986 (2.9K) [text/plain]
Saving to: ‘simple_inference.py’

simple_inference.py 100%[===================>]   2.92K  --.-KB/s    in 0s      

2023-10-04 03:04:08 (36.7 MB/s) - ‘simple_inference.py’ saved [2986/2986]



In [ ]:
# download the linear mse model path
!wget https://github.com/microsoft/LMOps/raw/main/promptist/aesthetic/sac%2Blogos%2Bava1-l14-linearMSE.pth

--2023-10-04 03:45:38--  https://github.com/microsoft/LMOps/raw/main/promptist/aesthetic/sac%2Blogos%2Bava1-l14-linearMSE.pth
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/microsoft/LMOps/main/promptist/aesthetic/sac%2Blogos%2Bava1-l14-linearMSE.pth [following]
--2023-10-04 03:45:38--  https://raw.githubusercontent.com/microsoft/LMOps/main/promptist/aesthetic/sac%2Blogos%2Bava1-l14-linearMSE.pth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3714759 (3.5M) [application/octet-stream]
Saving to: ‘sac+logos+ava1-l14-linearMSE.pth’

sac+logos+ava1-l14- 100%[===================>]   3.54M  --.-KB/s

In [ ]:
from datasets import load_dataset

dataset = load_dataset("poloclub/diffusiondb", name='2m_first_1k')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Print basic information about the dataset
# print("Number of splits:", dataset.num_splits)
print("List of splits:", dataset.keys())
print("Total number of examples in the train split:", len(dataset["train"]))

List of splits: dict_keys(['train'])
Total number of examples in the train split: 1000


In [ ]:
train_data = dataset['train']

In [ ]:
# train_data[0]['image']

In [ ]:
train_data['prompt'][:2]

['a renaissance portrait of dwayne johnson, art in the style of rembrandt!! intricate. ultra detailed, oil on canvas, wet - on - wet technique, pay attention to facial details, highly realistic, cinematic lightning, intricate textures, illusionistic detail, ',
 'portrait of a dancing eagle woman, beautiful blonde haired lakota sioux goddess, intricate, highly detailed art by james jean, ray tracing, digital painting, artstation, concept art, smooth, sharp focus, illustration, artgerm and greg rutkowski and alphonse mucha, vladimir kush, giger, 8 k ']

In [ ]:
print(len(train_data))

1000


In [ ]:
# for data in train_data['image'][:1]:
#   img_path = data
#   print(img_path)

for i in range(1):
  img_path = train_data[i]['image']
  print(img_path)

<PIL.PngImagePlugin.PngImageFile image mode=RGB size=512x768 at 0x796CBA5D5750>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
import simple_inference_custom
from simple_inference_custom import predict_aesthetic_score

In [ ]:
import time
import torch
import os

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

aesthetic_scores = []

# Initialize variables for timing
total_execution_time = 0
num_images = len(train_data[:5])  # Change this to the number of images you want to process

# for image_data in train_data[:5]:
for i in range(5):
    # Measure the start time
    start_time = time.time()

    # Load the image and move it to the GPU if available
    image_path = train_data[i]['image'].filename  # Replace with your image loading function

    if not os.path.exists(image_path):
        print(f"Image file not found: {image_path}")
        continue

    try:
        # Call the predict_aesthetic_score function from your custom script
        aesthetic_score = predict_aesthetic_score(image_path)
    except Exception as e:
        print(f"Error processing image: {image_path}")
        print(f"Error message: {str(e)}")
        continue

    # Measure the end time
    end_time = time.time()

    # Calculate the time taken for this image
    execution_time = end_time - start_time

    total_execution_time += execution_time

    aesthetic_scores.append(aesthetic_score)

# Calculate the average time per image
average_execution_time = total_execution_time / num_images

print(f"Total execution time for {num_images} images: {total_execution_time} seconds")
print(f"Average time per image: {average_execution_time} seconds")

Aesthetic score predicted by the model:
tensor([[5.9761]], device='cuda:0', grad_fn=<AddmmBackward0>)
Aesthetic score predicted by the model:
tensor([[7.0496]], device='cuda:0', grad_fn=<AddmmBackward0>)
Aesthetic score predicted by the model:
tensor([[5.0113]], device='cuda:0', grad_fn=<AddmmBackward0>)
Aesthetic score predicted by the model:
tensor([[6.2796]], device='cuda:0', grad_fn=<AddmmBackward0>)
Aesthetic score predicted by the model:
tensor([[5.2863]], device='cuda:0', grad_fn=<AddmmBackward0>)
Total execution time for 12 images: 57.45818591117859 seconds
Average time per image: 4.788182159264882 seconds


In [ ]:
# Extract float values from tensors in the aesthetic_scores list
aesthetic_scores_float = [score.item() for score in aesthetic_scores]

# Now, aesthetic_scores_float contains the float values
print(aesthetic_scores_float)

[5.976141929626465, 7.0495710372924805, 5.011256217956543, 6.279646873474121, 5.286251068115234]
